<a href="https://colab.research.google.com/github/chacha86/pythonai2/blob/main/deep_learning_exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import numpy as np
def ReLU(x) :
  return np.maximum(0, x)

def sigmoid(x) :
  return 1 / (1 + np.exp(-x))


In [17]:
class MyNet():
  def __init__(self, input, hidden, output):
    self.params = self.init_param(input, hidden, output)

  def init_param(self, input, hidden, output):
    params = {
        'w1' : self.get_rand_matrix(input, hidden),
        'b1' : np.zeros(hidden),
        'w2' : self.get_rand_matrix(hidden, output),
        'b2' : np.zeros(output)
    }

    return params

  def get_rand_matrix(self, m, n) :
    return np.random.randn(m, n) * 0.1

  def get_rand_vector(self, m) :
    return np.random.randn(m) - 1

  def forward_propagation(self, input_arr):
    z1 = np.dot(input_arr, self.params['w1']) + self.params['b1']
    o1 = ReLU(z1)
    z2 = np.dot(o1, self.params['w2']) + self.params['b2']
    o2 = sigmoid(z2)



    return o2

  def binary_cross_entropy(self, y_pred, y_true) :
    epsilon = 1e-15  # 로그의 결과가 무한대가 되는 것을 방지하기 위한 작은 값

    # 예측 값이 극단적인 값에 가까워지는 것을 방지
    y_pred = np.where(y_pred < epsilon, epsilon, y_pred)
    y_pred = np.where(y_pred > 1 - epsilon, 1 - epsilon, y_pred)
    return - (y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

  def get_loss_sum(self, train, target):
    errors = []
    for tr_row, tar_row in zip(train, target):
      pred = self.forward_propagation(tr_row)
      error = self.binary_cross_entropy(pred, tar_row)
      errors.append(error)

    return np.sum(errors)

  def numerical_diff(self, x, f):
    h = 1e-4

    if x.ndim == 1:
      x = x.reshape([1, x.size])
    grad = np.zeros_like(x)
    # it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    # while not it.finished:
    #     idx = it.multi_index
    #     tmp_val = x[idx]

    #     x[idx] = float(tmp_val) + h
    #     fxh1 = f(x)

    #     x[idx] = tmp_val - h
    #     fxh2 = f(x)

    #     grad[idx] = (fxh1 - fxh2) / (2*h)

    #     x[idx] = tmp_val
    #     it.iternext()

    for row in range(len(x)) :
      for col in range(len(x[row])) :
        tmp_val = x[(row, col)]
        x[(row, col)] = tmp_val + h
        fxh1 = f(x)
        x[(row, col)] = tmp_val - h
        fxh2 = f(x)

        grad[(row, col)] = (fxh1 - fxh2) / (2*h)
        x[(row, col)] = tmp_val

    return grad


  def gradient_descent(self, train, target):
    self.train = train
    self.target = target
    lr = 0.00001
    self.params['w1'] = self.params['w1'] - lr * self.numerical_diff(self.params['w1'], self.f)
    self.params['b1'] = self.params['b1'] - lr * self.numerical_diff(self.params['b1'], self.f)
    self.params['w2'] = self.params['w2'] - lr * self.numerical_diff(self.params['w2'], self.f)
    self.params['b2'] = self.params['b2'] - lr * self.numerical_diff(self.params['b2'], self.f)


  def f(self, x):
    return self.get_loss_sum(self.train, self.target)

In [18]:
import pandas as pd
fish = pd.read_csv('/content/Fish.csv')

target = fish['Species']
train = fish.drop('Species', axis=1)

target[target == 'Bream'] = 1
target[target != 1] = 0

<ipython-input-18-6f4cb124cb4c>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target[target == 'Bream'] = 1
<ipython-input-18-6f4cb124cb4c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target[target != 1] = 0


In [19]:
net = MyNet(2, 10, 1)

loss_list = []
for i in range(100):
  net.gradient_descent(train[['Length1', 'Weight']].values, target)
  loss_list.append(net.get_loss_sum(train[['Length1', 'Weight']].values, target))